# Step 0
## The REPL

In [1]:
def READ(program:str) -> str:
    return program

In [2]:
def EVAL(line:str) -> str:
    return line

In [3]:
def PRINT(line:str):
    print(line)

In [4]:
def rep(program):
    PRINT(EVAL(READ(program)))

#### Tests

In [5]:
rep(';=>%')

;=>%


# Step 1

In [6]:
# Mal types?

Symbol = str          
List   = list         
Number = (int, float)

#### Test

In [7]:
dgg = Number(9)
type(dgg)

TypeError: 'tuple' object is not callable

In [8]:
def read_str():
    tokens = tokenize(program)
    reader = Reader(tokens)
    read_form(reader)

In [20]:
class Reader:
    def __init__(self, tokens):
        self.tokens = tokens
        self.position = 0
        self.length = len(tokens)
    def nex(self):
        pos = self.position
        current_token = self.tokens[self.position]
        self.position += 1
        return current_token
    def peek(self):
        pos = self.position
        current_token = self.tokens[self.position]
        return current_token

In [21]:
import re

r = r'''[\s,]*(~@|[\[\]{}()'`~^@]|"(?:\\.|[^\\"])*"|;.*|[^\s\[\]{}('"`,;)]*)'''

def tokenize(program:str) -> list:
    tkns = re.split(r, program)
    tkns_list = []
    for t in tkns:
        if t != '':
            tkns_list.append(t) ### ESTO ESTA MAL
    return tkns_list

#### Tests

In [22]:
t = tokenize('( + 2 (* 3 4))')

t

['(', '+', '2', '(', '*', '3', '4', ')', ')']

tokenize

    [\s,]*: Matches any number of whitespaces or commas. This is not captured so it will be ignored and not tokenized. --- ANDA PERO ESTA MAL

    ~@: Captures the special two-characters ~@ (tokenized). ---- ANDA

    [\[\]{}()'`~^@]: Captures any special single character, one of []{}()'`~^@ (tokenized). ---- NO ANDA LOS PONE TODOS JUNTOS

    "(?:\\.|[^\\"])*"?: Starts capturing at a double-quote and stops at the next double-quote unless it was preceded by a backslash in which case it includes it until the next double-quote (tokenized). It will also match unbalanced strings (no ending double-quote) which should be reported as an error. ---- ANDA

    ;.*: Captures any sequence of characters starting with ; (tokenized). ---- ANDA

    [^\s\[\]{}('"`,;)]*: Captures a sequence of zero or more non special characters (e.g. symbols, numbers, "true", "false", and "nil") and is sort of the inverse of the one above that captures special characters (tokenized).


In [23]:
def read_form(tokens:Reader):
    tkn = tokens.peek()
    if tkn == '(':
        tokens.nex()
        t = read_list(tokens)
    else: 
        t = read_atom(tkn)
    return t


# TODO  The return value from read_form is a mal data type.

In [24]:
def read_list(tokens):
    result = []
    tkn = tokens.peek()
    while tkn != ')':
        t = read_form(tokens)
        result.append(t)
        tkn = tokens.nex()
    return result

# TODO poner algo que corte si te olvidas un parentesis de cierre 
# TODO si pones paren de cierre de mas, los ignora, corregir

This function will repeatedly call read_form with the Reader object until it encounters a ')' token (if it reach EOF before reading a ')' then that is an error). It accumulates the results into a List type. If your language does not have a sequential data type that can hold mal type values you may need to implement one (in types.qx). Note that read_list repeatedly calls read_form rather than read_atom. This mutually recursive definition between read_list and read_form is what allows lists to contain lists.


In [25]:
def read_atom(char):
    "Numbers become numbers; every other token is a symbol."
    try: return int(char)
    except ValueError:
        try: return float(char)
        except ValueError:
            return str(char)

In [26]:
ghh = read_atom('s')


#### Tests

In [30]:
my_str = "(ab(1 2)))))"

my_tok = Reader(my_str)
result = read_form(my_tok)

print(result)

['a', 'b', [1, ' ', 2, ')']]
